In [14]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from pprint import pprint

dbname = 'lr_luis_db'  
user = 'lorenzo_luis'  
password = 'password'  
host = 'localhost'  
port = '5432'  

try:
    connection = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    print(connection)
    cursor = connection.cursor()
    #table_name = 'employees'
    # Sentencia SQL para eliminar la tabla
    #drop_table_query = f'DROP TABLE IF EXISTS {table_name};'
    #cursor.execute(drop_table_query)

    cursor.execute("SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_name = 'employees');")
    table_exists = cursor.fetchone()[0]
    print("tabla existe ",table_exists)
    if not(table_exists):
        print("CREATING")
        cursor.execute('''
                       CREATE TABLE employees
                       ( employee_id INTEGER
                       , first_name VARCHAR(20)
                       , last_name VARCHAR(25)
                       , email VARCHAR(25)
                       , phone_number VARCHAR(20)
                       , hire_date DATE
                       , job_id VARCHAR(10)
                       , salary NUMERIC(8,2)
                       , commission_pct NUMERIC(2,2)
                       , manager_id INTEGER
                       , department_id INTEGER
                       ) ;
                       CREATE UNIQUE INDEX emp_emp_id_pk
                       ON employees (employee_id) ;
                       ALTER TABLE employees ADD
                       PRIMARY KEY (employee_id);
                       ''')
        cursor.execute('''
                       SELECT * FROM employees LIMIT 10;
                       SELECT count(1) FROM employees;
                       ''')
        connection.commit()
    else:
        engine_url = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'
        engine = create_engine(engine_url)

        df = pd.read_sql_query("SELECT * FROM employees;", engine)
        pprint(df.head())
        
        df = pd.read_sql_query("SELECT job_id FROM employees;", engine)
        pprint(df.head())
        
        
    
        connection.commit()
except Exception as e:
    print(f"Error al conectar a PostgreSQL: {e}")
finally:
   
    if connection:
        pass

<connection object at 0x000001E6A4A23780; dsn: 'user=lorenzo_luis password=xxx dbname=lr_luis_db host=localhost port=5432', closed: 0>
tabla existe  True
Empty DataFrame
Columns: [employee_id, first_name, last_name, email, phone_number, hire_date, job_id, salary, commission_pct, manager_id, department_id]
Index: []
Empty DataFrame
Columns: [job_id]
Index: []


El principio del codigo es la creacion de la conexion con pgadmin:
####
connection = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
cursor = connection.cursor()
####
Luego, se verifica que la no existencia de la base de datos, para no sobreescribir y que se ejecuten errores:
####
cursor.execute("SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_name = 'employees');")
table_exists = cursor.fetchone()[0]
if not(table_exists): ...
####
Despues, se ejecuta la siguiente accion:
####
cursor.execute('''
                       CREATE TABLE employees
                       ( employee_id INTEGER
                       , first_name VARCHAR(20)
                       , last_name VARCHAR(25)
                       , email VARCHAR(25)
                       , phone_number VARCHAR(20)
                       , hire_date DATE
                       , job_id VARCHAR(10)
                       , salary NUMERIC(8,2)
                       , commission_pct NUMERIC(2,2)
                       , manager_id INTEGER
                       , department_id INTEGER
                       ) ;
                       CREATE UNIQUE INDEX emp_emp_id_pk
                       ON employees (employee_id) ;
                       ALTER TABLE employees ADD
                       PRIMARY KEY (employee_id);
                       ''')
####
Esta parte se encarga de crear una tabla con el nombre  'employees', y crea las siguientes varaibles:
employee_id, first_name, last_name, email, phone_number , hire_date, job_id, salary ...
en donde al lado de cada nombre de las variables se declara el tipo y la longitud de esta:
INTEGER para variables discretas.
VARCHAR para strings.
DATE para fechas.
La otra parte de codigo:
####
CREATE UNIQUE INDEX emp_emp_id_pk
    ON employees (employee_id) ;
ALTER TABLE employees ADD
   PRIMARY KEY (employee_id);
####
se encarga de que la variable 'employee_id' no repita valores, siendo estos discretos.

In [15]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from pprint import pprint
import numpy as np

connection = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
cursor = connection.cursor()
cursor.execute("DROP TABLE IF EXISTS courses;")
cursor.execute(''' 
    CREATE TABLE courses
    (
        course_id SERIAL PRIMARY KEY,
        course_name VARCHAR(60),
        course_author VARCHAR(40),
        course_status VARCHAR(20),
        course_published_dt DATE      
    );
    
    CREATE UNIQUE INDEX crs_crs_id_pk
    ON courses (course_id);
''')
connection.commit()
engine_url = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'
engine = create_engine(engine_url)

df = pd.read_sql_query("SELECT * FROM courses;", engine)
pprint(df.head())

Empty DataFrame
Columns: [course_id, course_name, course_author, course_status, course_published_dt]
Index: []


In [87]:
for inserts in range(12):
    Course_Name=input("Course Name:  ")
    Course_author = input("Course author:  ")
    Course_status = input("Course status:   ")
    try:
        if Course_status!="draft":
            Date = input("Date :  ")
            cursor.execute(
                '''
                INSERT INTO courses ( course_name, course_author, course_status, course_published_dt)
                VALUES ( %s, %s, %s, %s)
                ''',
                ( Course_Name, Course_author, Course_status, Date)
            )
            
        else:
            cursor.execute(
                '''
                INSERT INTO courses ( course_name, course_author, course_status, course_published_dt)
                VALUES ( %s, %s, %s, %s)
                ''',
                ( Course_Name, Course_author, Course_status, None)
            )
            
        connection.commit()
        print("tudo ben")
    except Exception as e:
        connection.rollback()
        print(f"Error inserting data: {e}")
    
    

Error inserting data: invalid input syntax for type date: ""
LINE 3:                 VALUES ( '', '', '', '')
                                             ^

Error inserting data: invalid input syntax for type date: ""
LINE 3:                 VALUES ( '', '', '', '')
                                             ^

Error inserting data: invalid input syntax for type date: ""
LINE 3:                 VALUES ( '', '', '', '')
                                             ^

Error inserting data: invalid input syntax for type date: ""
LINE 3:                 VALUES ( '', '', '', '')
                                             ^

Error inserting data: invalid input syntax for type date: ""
LINE 3:                 VALUES ( '', '', '', '')
                                             ^

Error inserting data: invalid input syntax for type date: ""
LINE 3:                 VALUES ( '', '', '', '')
                                             ^

Error inserting data: invalid input syntax for type 

In [17]:
inf=[
     
    ["1",    "Programming using Python",         "Bob Dillon",     "published","2020-09-30"],
    ["2",    "Data Engieering using Python",     "Bob Dillon",     "published","2020-07-15"],
    ["3",    "Data Engineering using Scala",     "Elvis Presley",  "draft"],
    ["4",    "Programming using Scala",          "Elvis Presley",  "published","2020-05-12"],
    ["5",    "Programming using Java",           "Mike Jack",      "inactive","2020-08-10"],
    ["6",    "Web Applications-Python Flask",    "Bob Dillon",     "inactive","2020-07-20"],
    ["7",    "Web Applications-Java Spring"  ,   "Mike Jack " ,    "draft"],
    ["8"     "Pipleine Orchestration -Python",   "Bob Dillon",     "draft"],
    ["9",    "Streaming Pipelines-Python",       "Bob Dillon",     "published","2020-09-30"],
    ["10",   "Web Applications-Scala Play",      "Elvis Presley",  "inactive","2020-06-23"],
    ["11",   "Web Applications-Python Django",   "Bob Dillon",     "published","2020-07-05"],
]
for line in inf:
    try:
        if line[3]=="draft":
            cursor.execute(
                    '''
                    INSERT INTO courses ( course_name, course_author, course_status, course_published_dt)
                    VALUES ( %s, %s, %s, %s)
                    ''',
                    ( line[1], line[2], line[3],None)
                )
        else:
            cursor.execute(
                    '''
                    INSERT INTO courses ( course_name, course_author, course_status, course_published_dt)
                    VALUES ( %s, %s, %s, %s)
                    ''',
                    ( line[1], line[2], line[3],line[4])
                )
        connection.commit()
        print("tudo ben")
    except Exception as e:
        connection.rollback()
        print(f"Error inserting data: {e}")
        
    

tudo ben
tudo ben
tudo ben
tudo ben
tudo ben
tudo ben
tudo ben
Error inserting data: list index out of range
tudo ben
tudo ben
tudo ben


In [18]:
engine_url = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'
engine = create_engine(engine_url)

df = pd.read_sql_query("SELECT * FROM courses;", engine)
pprint(df)

    course_id                     course_name  course_author course_status  \
0           1        Programming using Python     Bob Dillon     published   
1           2    Data Engieering using Python     Bob Dillon     published   
2           3    Data Engineering using Scala  Elvis Presley         draft   
3           4         Programming using Scala  Elvis Presley     published   
4           5          Programming using Java      Mike Jack      inactive   
5           6   Web Applications-Python Flask     Bob Dillon      inactive   
6           7    Web Applications-Java Spring     Mike Jack          draft   
7           8      Streaming Pipelines-Python     Bob Dillon     published   
8           9     Web Applications-Scala Play  Elvis Presley      inactive   
9          10  Web Applications-Python Django     Bob Dillon     published   
10         11        Programming using Python     Bob Dillon     published   
11         12    Data Engieering using Python     Bob Dillon    

In [19]:
try:
    cursor.execute("DELETE FROM courses where course_status = 'inactive';")
    connection.commit()
    engine_url = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'
    engine = create_engine(engine_url)

    df = pd.read_sql_query("SELECT * FROM courses;", engine)
    pprint(df)

except psycopg2.DatabaseError as e:
    connection.rollback()  # Rollback en caso de error
    print("Error executing SQL query:", e)


    course_id                     course_name  course_author course_status  \
0           1        Programming using Python     Bob Dillon     published   
1           2    Data Engieering using Python     Bob Dillon     published   
2           3    Data Engineering using Scala  Elvis Presley         draft   
3           4         Programming using Scala  Elvis Presley     published   
4           7    Web Applications-Java Spring     Mike Jack          draft   
5           8      Streaming Pipelines-Python     Bob Dillon     published   
6          10  Web Applications-Python Django     Bob Dillon     published   
7          11        Programming using Python     Bob Dillon     published   
8          12    Data Engieering using Python     Bob Dillon     published   
9          13    Data Engineering using Scala  Elvis Presley         draft   
10         14         Programming using Scala  Elvis Presley     published   
11         17    Web Applications-Java Spring     Mike Jack     

In [20]:
try:
    cursor.execute('''
                SELECT COUNT(DISTINCT course_author) FROM courses;
                ''')
    
    print("El total de autores distintos es de ",cursor.fetchone()[0])
    engine_url = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'
    engine = create_engine(engine_url)

    df = pd.read_sql_query("SELECT * FROM courses ORDER by course_author;", engine)
    pprint(df)  
    
    
except psycopg2.DatabaseError as e:
    connection.rollback()  # Rollback en caso de error
    print("Error executing SQL query:", e)    

El total de autores distintos es de  3
    course_id                     course_name  course_author course_status  \
0          20  Web Applications-Python Django     Bob Dillon     published   
1           2    Data Engieering using Python     Bob Dillon     published   
2           8      Streaming Pipelines-Python     Bob Dillon     published   
3          10  Web Applications-Python Django     Bob Dillon     published   
4          11        Programming using Python     Bob Dillon     published   
5          12    Data Engieering using Python     Bob Dillon     published   
6          18      Streaming Pipelines-Python     Bob Dillon     published   
7           1        Programming using Python     Bob Dillon     published   
8           3    Data Engineering using Scala  Elvis Presley         draft   
9           4         Programming using Scala  Elvis Presley     published   
10         13    Data Engineering using Scala  Elvis Presley         draft   
11         14         Pro

VI)

In [23]:
dbname = 'lr_luis_db'  
user = 'lorenzo_luis'  
password = 'password'  
host = 'localhost'  
port = '5432' 

In [24]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from pprint import pprint
import numpy as np


connection = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
cursor = connection.cursor()
cursor.execute("DROP TABLE IF EXISTS users;")
cursor.execute('''
               CREATE TABLE users(
                    user_id SERIAL PRIMARY KEY,
                    user_first_name VARCHAR(30),
                    user_last_name VARCHAR(30),
                    user_email_id VARCHAR(50),
                    user_gender VARCHAR(1),
                    user_unique_id VARCHAR(15),
                    user_phone_no VARCHAR(20),
                    user_dob DATE,
                    created_ts TIMESTAMP
                )
               ''')
cursor.execute('''
               INSERT INTO users (
    user_first_name, user_last_name, user_email_id, user_gender, 
    user_unique_id, user_phone_no, user_dob, created_ts
) VALUES
    ('Giuseppe', 'Bode', 'gbode0@imgur.com', 'M', '88833-8759', 
     '+86 (764) 443-1967', '1973-05-31', '2018-04-15 12:13:38'),
    ('Lexy', 'Gisbey', 'lgisbey1@mail.ru', 'F', '262501-029', 
     '+86 (751) 160-3742', '2003-05-31', '2020-12-29 06:44:09'),
    ('Karel', 'Claringbold', 'kclaringbold2@yale.edu', 'F', '391-33-2823', 
     '+62 (445) 471-2682', '1985-11-28', '2018-11-19 00:04:08'),
    ('Marv', 'Tanswill', 'mtanswill3@dedecms.com', 'F', '1195413-80', 
     '+62 (497) 736-6802', '1998-05-24', '2018-11-19 16:29:43'),
    ('Gertie', 'Espinoza', 'gespinoza4@nationalgeographic.com', 'M', '471-24-6869', 
     '+249 (687) 506-2960', '1997-10-30', '2020-01-25 21:31:10'),
    ('Saleem', 'Danneil', 'sdanneil5@guardian.co.uk', 'F', '192374-933', 
     '+63 (810) 321-0331', '1992-03-08', '2020-11-07 19:01:14'),
    ('Rickert', 'O''Shiels', 'roshiels6@wikispaces.com', 'M', '749-27-47-52', 
     '+86 (184) 759-3933', '1972-11-01', '2018-03-20 10:53:24'),
    ('Cybil', 'Lissimore', 'clissimore7@pinterest.com', 'M', '461-75-4198', 
     '+54 (613) 939-6976', '1978-03-03', '2019-12-09 14:08:30'),
    ('Melita', 'Rimington', 'mrimington8@mozilla.org', 'F', '892-36-676-2', 
     '+48 (322) 829-8638', '1995-12-15', '2018-04-03 04:21:33'),
    ('Benetta', 'Nana', 'bnana9@google.com', 'M', '197-54-1646', 
     '+420 (934) 611-0020', '1971-12-07', '2018-10-17 21:02:51'),
    ('Gregorius', 'Gullane', 'ggullanea@prnewswire.com', 'F', '232-55-52-58', 
     '+62 (780) 859-1578', '1973-09-18', '2020-01-14 23:38:53'),
    ('Una', 'Glayzer', 'uglayzerb@pinterest.com', 'M', '898-84-336-6', 
     '+380 (840) 437-3981', '1983-05-26', '2019-09-17 03:24:21'),
    ('Jamie', 'Vosper', 'jvosperc@umich.edu', 'M', '247-95-68-44', 
     '+81 (205) 723-1942', '1972-03-18', '2020-07-23 16:39:33'),
    ('Calley', 'Tilson', 'ctilsond@issuu.com', 'F', '415-48-894-3', 
     '+229 (698) 777-4904', '1987-06-12', '2020-06-05 12:10:50'),
    ('Peadar', 'Gregorowicz', 'pgregorowicze@omniture.com', 'M', '403-39-5-869', 
     '+7 (267) 853-3262', '1996-09-21', '2018-05-29 23:51:31'),
    ('Jeanie', 'Webling', 'jweblingf@booking.com', 'F', '399-83-05-03', 
     '+351 (684) 413-0550', '1994-12-27', '2018-02-09 01:31:11'),
    ('Yankee', 'Jelf', 'yjelfg@wufoo.com', 'F', '607-99-0411', 
     '+1 (864) 112-7432', '1988-11-13', '2019-09-16 16:09:12'),
    ('Blair', 'Aumerle', 'baumerleh@toplist.cz', 'F', '430-01-578-5', 
     '+7 (393) 232-1860', '1979-11-09', '2018-10-28 19:25:35'),
    ('Pavlov', 'Steljes', 'psteljesi@macromedia.com', 'F', '571-09-6181', 
     '+598 (877) 881-3236', '1991-06-24', '2020-09-18 05:34:31'),
    ('Darn', 'Hadeke', 'dhadekej@last.fm', 'M', '478-32-02-87', 
     '+370 (347) 110-4270', '1984-09-04', '2018-02-10 12:56:00'),
    ('Wendell', 'Spanton', 'wspantonk@de.vu', 'F', null, 
     '+84 (301) 762-1316', '1973-07-24', '2018-01-30 01:20:11'),
    ('Carlo', 'Yearby', 'cyearbyl@comcast.net', 'F', null, 
     '+55 (288) 623-4067', '1974-11-11', '2018-06-24 03:18:40'),
    ('Sheila', 'Evitts', 'sevittsm@webmd.com', null, '830-40-5287',
     null, '1977-03-01', '2020-07-20 09:59:41'),
    ('Sianna', 'Lowdham', 'slowdhamn@stanford.edu', null, '778-0845', 
     null, '1985-12-23', '2018-06-29 02:42:49'),
    ('Phylys', 'Aslie', 'paslieo@qq.com', 'M', '368-44-4478', 
     '+86 (765) 152-8654', '1984-03-22', '2019-10-01 01:34:28')
               ''')
connection.commit()

In [25]:
engine_url = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'
engine = create_engine(engine_url)

df = pd.read_sql_query("SELECT * FROM users;", engine)
print(df.user_first_name[24]) 

Phylys


In [26]:
try:
    cursor.execute('''
                SELECT EXTRACT(YEAR FROM user_dob) as year, COUNT(*) as count_years
                FROM users
                GROUP BY year
                ORDER BY year DESC;
                ''')
    
    dicc={"created_year":[],"user_count":[]}
    for row in cursor.fetchall():
        dicc["created_year"].append(row[0])
        dicc["user_count"].append(row[1])
    dataframe = pd.DataFrame(dicc)
    pprint(dataframe)
except psycopg2.DatabaseError as e:
    connection.rollback()  # Rollback en caso de error
    print("Error executing SQL query:", e)   

   created_year  user_count
0          2003           1
1          1998           1
2          1997           1
3          1996           1
4          1995           1
5          1994           1
6          1992           1
7          1991           1
8          1988           1
9          1987           1
10         1985           2
11         1984           2
12         1983           1
13         1979           1
14         1978           1
15         1977           1
16         1974           1
17         1973           3
18         1972           2
19         1971           1


XIII)

In [27]:
try:
    cursor.execute('''
                SELECT user_id, user_dob, user_email_id, 
                        TO_CHAR(user_dob, 'Dy') as day
                FROM users
                WHERE EXTRACT(MONTH FROM user_dob) = 5
                ORDER BY user_dob ;
                ''')
    
    dicc={"user_id":[],
          "user_dob":[],
          "user_email":[],
          "user_day_of_birth":[]}
    for row in cursor.fetchall():
        dicc["user_id"].append(row[0])
        dicc["user_dob"].append(row[1])
        dicc["user_email"].append(row[2])
        dicc["user_day_of_birth"].append(row[3])
    dataframe = pd.DataFrame(dicc)
    pprint(dataframe)
    
        
except psycopg2.DatabaseError as e:
    connection.rollback()  # Rollback en caso de error
    print("Error executing SQL query:", e)   

   user_id    user_dob               user_email user_day_of_birth
0        1  1973-05-31         gbode0@imgur.com               Thu
1       12  1983-05-26  uglayzerb@pinterest.com               Thu
2        4  1998-05-24   mtanswill3@dedecms.com               Sun
3        2  2003-05-31         lgisbey1@mail.ru               Sat


IX)

In [28]:
try:
    cursor.execute('''
                SELECT user_id, user_email_id, created_ts, 
                       user_first_name || ' ' || user_last_name as user_name,
                       EXTRACT(YEAR FROM created_ts) AS year
                FROM users
                WHERE EXTRACT(YEAR FROM created_ts) = 2019
                ORDER BY user_name ;
                ''')
    
    dicc={"user_id":[],
          "user_name":[],
          "user_email":[],
          "created_ts":[],
          "created_year":[]}
    
    for row in cursor.fetchall():
        
        dicc["user_id"].append(row[0])
        dicc["user_name"].append(row[3])
        dicc["user_email"].append(row[1])
        dicc["created_ts"].append(row[2])
        dicc["created_year"].append(row[4])
        
    dataframe = pd.DataFrame(dicc)
    pprint(dataframe)
    
        
except psycopg2.DatabaseError as e:
    connection.rollback()  # Rollback en caso de error
    print("Error executing SQL query:", e)   

   user_id        user_name                 user_email          created_ts  \
0        8  Cybil Lissimore  clissimore7@pinterest.com 2019-12-09 14:08:30   
1       25     Phylys Aslie             paslieo@qq.com 2019-10-01 01:34:28   
2       12      Una Glayzer    uglayzerb@pinterest.com 2019-09-17 03:24:21   
3       17      Yankee Jelf           yjelfg@wufoo.com 2019-09-16 16:09:12   

  created_year  
0         2019  
1         2019  
2         2019  
3         2019  


X)

In [62]:
try:
    cursor.execute('''
                SELECT user_gender as genres, COUNT(*) as user_count
                FROM users
                GROUP BY genres
                ORDER BY user_count DESC;
                ''')
    
    dicc={"user_gender":[],
          "user_count":[]}
    
    for row in cursor.fetchall():
        
        dicc["user_gender"].append("Male" if row[0] == "M" else ("Female" if row[0] == "F" else "Not specified"))
        dicc["user_count"].append(row[1])

        
    dataframe = pd.DataFrame(dicc)
    pprint(dataframe)
    
        
except psycopg2.DatabaseError as e:
    connection.rollback()  # Rollback en caso de error
    print("Error executing SQL query:", e)   

     user_gender  user_count
0         Female          13
1           Male          10
2  Not specified           2


XI)

In [73]:
try:
    cursor.execute('''
                SELECT user_id,  user_unique_id ,RIGHT(user_unique_id, 4) AS last_four_digits
                FROM users
                ''')
    
    dicc={"user_id":[],
          "user_unique_id":[],
          "user_unique_id_last_4":[]}
    for row in cursor.fetchall():
        
        dicc["user_id"].append(row[0])
        dicc["user_unique_id"].append(row[1])
        dicc["user_unique_id_last_4"].append(row[2])
    dataframe = pd.DataFrame(dicc)
    pprint(dataframe)      
except psycopg2.DatabaseError as e:
    connection.rollback()  # Rollback en caso de error
    print("Error executing SQL query:", e)   

    user_id user_unique_id user_unique_id_last_4
0         1     88833-8759                  8759
1         2     262501-029                  -029
2         3    391-33-2823                  2823
3         4     1195413-80                  3-80
4         5    471-24-6869                  6869
5         6     192374-933                  -933
6         7   749-27-47-52                  7-52
7         8    461-75-4198                  4198
8         9   892-36-676-2                  76-2
9        10    197-54-1646                  1646
10       11   232-55-52-58                  2-58
11       12   898-84-336-6                  36-6
12       13   247-95-68-44                  8-44
13       14   415-48-894-3                  94-3
14       15   403-39-5-869                  -869
15       16   399-83-05-03                  5-03
16       17    607-99-0411                  0411
17       18   430-01-578-5                  78-5
18       19    571-09-6181                  6181
19       20   478-32

XII)

In [87]:
try:
    cursor.execute('''
                SELECT LEFT(user_phone_no,3) as three_digits, COUNT(*) as user_count
                FROM users
                GROUP BY three_digits
                ORDER BY user_count;
                ''')
    
    dicc={"cpuntry_code":[],
          "user_count":[]}
    
    for row in cursor.fetchall():

        dicc["country_code"].append(row[0][1:] if row[0]!=None else None )
        dicc["user_count"].append(row[1])

        
    dataframe = pd.DataFrame(dicc)
    pprint(dataframe)
    
        
except psycopg2.DatabaseError as e:
    connection.rollback()  # Rollback en caso de error
    print("Error executing SQL query:", e)  

   user_gender  user_count
0           22           1
1           81           1
2           1            1
3           37           1
4           63           1
5           84           1
6           55           1
7           38           1
8           24           1
9           54           1
10          35           1
11          42           1
12          59           1
13          48           1
14          7            2
15        None           2
16          62           3
17          86           4
